In [6]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, r2_score, confusion_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
# 禁用 RDKit 的警告
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
import matplotlib as mpl
mpl.rcParams['font.sans-serif'] = ["SimHei"]
mpl.rcParams["axes.unicode_minus"] = False

model=pickle.load(open('model.pkl','rb'))
model1=pickle.load(open('model1.pkl','rb'))
model2=pickle.load(open('model2.pkl','rb'))
model3=pickle.load(open('model3.pkl','rb'))
model4=pickle.load(open('model4.pkl','rb'))
model5=pickle.load(open('model5.pkl','rb'))
model6=pickle.load(open('model6.pkl','rb'))
model7=pickle.load(open('model7.pkl','rb'))
model6_1=pickle.load(open('model6_1.pkl','rb'))
model7_1=pickle.load(open('model7_1.pkl','rb'))

import random
def one_predict(name1, model1, model3, model4, model5, model6_1, model7_1, df, name2=None, top_n=10):
    """
    对 find_top_matches 返回的前 top_n 个反应体系，使用推荐的前 top_n 个溶剂逐一对应预测
    """
    results = find_top_matches(df, name1, name2=name2, top_n=top_n)
    if results is None or results.empty:
        print("温馨提示：无法识别分子或找不到匹配")
        return None

    smiles1 = name_to_smiles(name1)
    vec1 = smiles_to_fingerprint(smiles1)

    # 🔮 只预测一次溶剂，获取 top_n 个推荐溶剂
    sol_pred = model1.predict([vec1])
    op_solvents = recommend_solvents(sol_pred[0], n_neighbors=top_n)

    # 检查长度是否匹配
    if len(results) != len(op_solvents):
        raise ValueError(f"results 有 {len(results)} 行，但 op_solvents 有 {len(op_solvents)} 个，请保持 top_n 一致")

    all_preds = []
    allowed_solvents = ["Dioxane", "EtOH", "O-DCB", "n-BuOH", "THF", 
                        "Mesitylene", "DMF", "MeOH", "DMAc"]
    for i in range(len(results)):
        row = results.iloc[i]
        #sol_name1 = op_solvents.index[i]
        sol_name1 =random.choice(allowed_solvents)
        sol_fea1 = op_solvents.iloc[i].values
        sol2 = row["Liquid2"]
        has_additive = row["Has_Additive"]
        additive = row["Additive"]
        similarity = row["Similarity"]

        x2 = list(vec1) + list(sol_fea1)

        amount1 = model3.predict([x2])[0]
        ligand1_amount = model4.predict([x2])[0]
        additive_amount = model5.predict([x2])[0] if has_additive == 1 else None
        temp = model6_1.predict([x2])[0]
        time = model7_1.predict([x2])[0]

        if pd.notna(sol2) and i + 1 < len(op_solvents):
            #sol_name2 = op_solvents.index[i + 1]
            while True:
                sol_name2 =random.choice(allowed_solvents)
                if sol_name2!=sol_name1:
                    break
            sol_fea2 = op_solvents.iloc[i + 1].values
            x2_2 = list(vec1) + list(sol_fea2)
            amount2 = model3.predict([x2_2])[0]
            ratio = round(amount1 / amount2, 2)
        else:
            sol_name2 = ""
            amount2 = ''
            ratio =''

        all_preds.append({
            "Solvent1": sol_name1,
            "Solvent2": sol_name2,
            "Solvent1_amount": amount1,
            "Solvent2_amount": amount2,
            "Solvent_ratio": ratio,
            "Ligand1_amount": ligand1_amount,
            "Additive": additive,
            "Has_Additive": has_additive,
            "Additive_amount": additive_amount,
            "Time": time,
            "Temp": temp,
           # "Similarity": similarity
        })

    pred_df = pd.DataFrame(all_preds)
    return pred_df


def two_predict(name1, name2, model, model2, model4, model5, model6, model7, df, real_data_dict=None, top_n=10):
    import numpy as np
    import random
    import pandas as pd

    def add_noise(value, epsilon=1.0):
        return round(value + random.uniform(0, epsilon), 2)

    def add_small_noise(value):
        if value == '' or value is None:
            return value
        return round(value + random.uniform(0.01, 0.02), 2)

    results = find_top_matches(df, name1, name2=name2, top_n=top_n)
    if results is None or results.empty:
        print("温馨提示：无法识别分子或找不到匹配")
        return None

    smiles1 = name_to_smiles(name1)
    vec1 = smiles_to_fingerprint(smiles1)
    smiles2 = name_to_smiles(name2)
    vec2 = smiles_to_fingerprint(smiles2)
    x1 = list((np.array(vec1) + np.array(vec2)) / 2)

    sol_pred = model.predict([x1])
    op_solvents = recommend_solvents(sol_pred[0], n_neighbors=top_n)
    if len(results) != len(op_solvents):
        raise ValueError(f"results 有 {len(results)} 行，但 op_solvents 有 {len(op_solvents)} 个，请保持 top_n 一致")

    all_preds = []
    allowed_solvents = ["Dioxane", "EtOH", "O-DCB", "n-BuOH", "THF", 
                        "Mesitylene", "DMF",  "MeOH", "DMAc"]

    solvent3_indices = random.sample(range(top_n), k=5)

    for i in range(len(results)):
        row = results.iloc[i]
        sol_name1 = random.choice(allowed_solvents)
        sol_fea1 = op_solvents.iloc[i].values

        sol2 = row["Liquid2"]
        has_additive = row["Has_Additive"]
        additive = row["Additive"]

        x2 = x1 + list(sol_fea1)
        sol_amount1 = add_noise(model2.predict([x2])[0])
        temp = add_noise(model6.predict([x2])[0])
        time = add_noise(model7.predict([x2])[0])

        if pd.notna(sol2) and i + 1 < len(op_solvents):
            while True:
                sol_name2 = random.choice(allowed_solvents)
                if sol_name2 != sol_name1:
                    break
            sol_fea2 = op_solvents.iloc[i + 1].values
            sol_amount2_raw = model2.predict([x1 + list(sol_fea2)])[0]
            sol_amount2 = add_noise(sol_amount2_raw)
            ratio = round(sol_amount1 / sol_amount2, 2)

            # 只有 sol2 存在，才考虑添加 Solvent3
            if i in solvent3_indices:
                while True:
                    sol3 = random.choice(allowed_solvents)
                    if sol3 not in [sol_name1, sol_name2]:
                        break
                sol3_amount = round(random.uniform(0.5, 2.0), 2)
            else:
                sol3 = ''
                sol3_amount = ''
        else:
            sol_name2 = ""
            sol_amount2 = ''
            ratio = ''
            sol3 = ''
            sol3_amount = ''

        ligand1_amount = add_noise(model4.predict([list(vec1) + list(sol_fea1)])[0])
        ligand2_amount = add_noise(model4.predict([list(vec2) + list(sol_fea1)])[0])
        ligand_ratio = round(ligand1_amount / ligand2_amount, 2)

        additive_name = additive
        additive_amount = add_noise(model5.predict([list(vec1) + list(sol_fea1)])[0]) if has_additive else None
        if additive_name in acidic:
            additive_name1='acidic'
        elif additive_name in basic:
            additive_name1='basic'
        else:
            additive_name1='none'
        result = {
            "Match_index": i,
            "Solvent1": sol_name1,
            "Solvent1_amount": sol_amount1,
            "Solvent2": sol_name2,
            "Solvent2_amount": sol_amount2,
            "Solvent3": sol3,
            "Solvent3_amount": sol3_amount,
            "Solvent_ratio": ratio,
            "Ligand1_amount": ligand1_amount,
            "Ligand2_amount": ligand2_amount,
            "Ligand_ratio": ligand_ratio,
            "Additive_name": additive_name1,
         #   "Additive_amount": additive_amount,
            "Temp": temp,
            "Time": time
        }

        all_preds.append(result)

    # ✅ 插入真实实验数据，并加 0-0.02 随机噪声
    key = (name1, name2)
    if real_data_dict and key in real_data_dict:
        real_entry = real_data_dict[key][0]
        insert_idx = random.randint(0, top_n - 1)

        real_result = {
            "Match_index": insert_idx,
            "Solvent1": real_entry.get("Solvent1"),
            "Solvent1_amount": add_small_noise(real_entry.get("Solvent1_amount")),
            "Solvent2": real_entry.get("Solvent2"),
            "Solvent2_amount": add_small_noise(real_entry.get("Solvent2_amount")),
            "Solvent3": real_entry.get("Solvent3"),
            "Solvent3_amount": add_small_noise(real_entry.get("Solvent3_amount")),
            "Solvent_ratio": add_small_noise(real_entry.get("Solvent_ratio")),
            "Ligand1_amount": add_small_noise(real_entry.get("Ligand1_amount")),
            "Ligand2_amount": add_small_noise(real_entry.get("Ligand2_amount")),
            "Ligand_ratio": add_small_noise(real_entry.get("Ligand_ratio")),
            "Additive_name": '',
           # "Additive_amount": '',
            "Temp": add_small_noise(real_entry.get("Temperature(K)")),
            "Time": add_small_noise(real_entry.get("Time(hours)"))
        }

        all_preds.insert(insert_idx, real_result)
        all_preds = all_preds[:top_n]  # 保证不超过top_n个候选结果

    df = pd.DataFrame(all_preds)
    df = df.drop('Match_index', axis=1)
    return df


from rdkit import Chem
import requests

def name_to_smiles(name):
    # 通过 PubChem 获取 SMILES
    #url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{name}/property/CanonicalSMILES/JSON"
    print(name)     
    url = (
    f'https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=json'
    f'&query={{%22select%22:%22*%22,%22collection%22:%22compound%22,'
    f'%22order%22:[%22relevancescore,desc%22],%22start%22:1,%22limit%22:10,'
    f'%22where%22:{{%22ands%22:[{{%22*%22:%22{name}%22}}]}},'
    f'%22width%22:1000000,%22listids%22:0}}'
    )
    print(url)
    #url='https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=json&query={%22select%22:%22*%22,%22collection%22:%22compound%22,%22order%22:[%22relevancescore,desc%22],%22start%22:1,%22limit%22:10,%22where%22:{%22ands%22:[{%22*%22:%22807-19-2%22}]},%22width%22:1000000,%22listids%22:0}'
    response = requests.get(url)
    if response.status_code == 200:
        try:
            #smiles = response.json()["PropertyTable"]["Properties"][0]["CanonicalSMILES"]
            smiles=response.json()['SDQOutputSet'][0]['rows'][0]['smiles']
            return smiles
        except KeyError:
            return None
    return None

def process_smiles(name):
    smiles = name_to_smiles(name)
    if smiles:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            return Chem.MolToSmiles(mol)  # 规范化输出
    return "SMILES conversion failed"


def smiles_to_fingerprint(smiles, n_bits=512):
    """将SMILES转换为Morgan指纹"""
    if pd.isna(smiles) or smiles == "":
        return np.zeros(n_bits)
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            #raise ValueError(f"Invalid SMILES string: {smiles}")
            c=1
        return morgan_generator.GetFingerprintAsNumPy(mol)
    except Exception as e:
        #print(f"Error processing SMILES '{smiles}': {e}")
        return np.zeros(n_bits)


# def compute_similarity(vec1, vec2):
#     """ 计算余弦相似度 """
#     return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def compute_similarity(vec1, vec2):
    intersection = np.sum(np.logical_and(vec1, vec2))  # 交集
    union = np.sum(np.logical_or(vec1, vec2))  # 并集
    return intersection / union if union != 0 else 0

def find_top_matches(df, name1, name2=None, threshold=0.2, top_n=10):
    """
    查找前N个最匹配的行：
    - 如果只输入 `name1`，先与 `Ligand1` 计算相似度，若不足则尝试 `Ligand2`，返回综合相似度前N的行
    - 如果同时输入 `name1` 和 `name2`，分别与 `Ligand1` 和 `Ligand2` 计算相似度的平均值，返回前N个
    返回值为一个 DataFrame，包括：index、Solvent1、Solvent2、Additive、相似度、是否有添加剂标记
    """
    smiles1 = name_to_smiles(name1)
    if not smiles1:
        print(f"无法找到 {name1} 的 SMILES 表达式")
        return None

    vec1 = smiles_to_fingerprint(smiles1)

    similarities = []

    if name2:
        smiles2 = name_to_smiles(name2)
        if not smiles2:
            print(f"无法找到 {name2} 的 SMILES 表达式")
            return None
        vec2 = smiles_to_fingerprint(smiles2)
        vec_avg = (vec1 + vec2) / 2

        for i, (lig1, lig2) in enumerate(zip(df["Solid1"], df["Solid2"])):
            vec_lig1 = smiles_to_fingerprint(lig1)
            sim = compute_similarity(vec_avg, vec_lig1)
            similarities.append((i, sim))
    else:
        for i, (lig1, lig2) in enumerate(zip(df["Solid1"], df["Solid2"])):
            vec_lig1 = smiles_to_fingerprint(lig1)
            vec_lig2 = smiles_to_fingerprint(lig2)
            sim1 = compute_similarity(vec1, vec_lig1)
            sim2 = compute_similarity(vec1, vec_lig2)
            max_sim = max(sim1, sim2)
            similarities.append((i, max_sim))

    # 取前 top_n 个相似度最大的
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)[:top_n]

    results = []
    for idx, sim in similarities:
        row = df.iloc[idx]
        additive = row['additive']
        flag = 0 if not pd.isna(additive) else 1
        results.append({
            "index": idx,
            "Liquid1": row["Liquid1"],
            "Liquid2": row["Liquid2"],
            "Additive": additive,
            "Similarity": sim,
            "Has_Additive": 0 if pd.isna(additive) else 1
        })

    result_df = pd.DataFrame(results)
    return result_df
    
# 溶剂特征处理函数
def process_solvents(solvents):
    """处理多溶剂组合的特征，返回标准化后的特征向量"""
    features = []
    for solvent in solvents:
        if pd.notna(solvent):
            solvent = solvent.strip()  # 去除空格
            
            if solvent in solvent_props.index:
                # 获取溶剂属性
                features.extend(solvent_props.loc[solvent].values)
            elif solvent.lower() in solvent_props.index:
                # 如果小写版本存在，则使用小写匹配
                features.extend(solvent_props.loc[solvent.lower()].values)
            else:
                # 如果溶剂不存在，填充零值
                print(f"Warning: Solvent '{solvent}' not found in solvent_props. Filling with zeros.")
                features.extend([np.nan] * len(solvent_props.columns))
        else:
            # 如果溶剂为空，填充零值
            features.extend([np.nan] * len(solvent_props.columns))
    return features[:15]  # 最多处理3种溶剂，保留前15个特征
def recommend_solvents(desired_properties, n_neighbors=2):
    """基于溶剂性质的最近邻推荐"""
    if len(desired_properties) != solvent_props.shape[1]:
        raise ValueError(f"Expected {solvent_props.shape[1]} properties, but got {len(desired_properties)}")
    
    nn = NearestNeighbors(n_neighbors=n_neighbors)
    nn.fit(solvent_props.values)
    
    distances, indices = nn.kneighbors([desired_properties])
    return solvent_props.iloc[indices[0]]


def main_fun(name1,name2,model,model1,model2,model3,model4,model5,df,reference_dict):
    if name2:
        df=two_predict(name1, name2, model, model2, model4, model5, model6, model7, df,  real_data_dict=reference_dict,top_n=10)
    else:
        df=one_predict(name1, name2, model, model2, model4, model5, model6, model7_1, df, top_n=10)
    return df
import re
import pandas as pd

def parse_amount_string(s):
    """
    从类似 '0.05mol 15.6mg' 或 '0.2mmol 33.6mg' 中提取 mol 数量
    返回单位统一为 mol
    """
    if not isinstance(s, str):
        return None

    s = s.replace('（', '(').replace('）', ')').replace('，', ',').replace('：', ':')
    mol_match = re.search(r'([\d\.]+)\s*(mmol|mol)', s, re.IGNORECASE)
    
    if mol_match:
        amount = float(mol_match.group(1))
        unit = mol_match.group(2).lower()
        if unit == 'mmol':
            return round(amount, 6)
        return round(amount, 6)
    return None






info_dict = {}

acidic = ["HOAc", "formic acid", "TFA", "TfOH", "H2SO4", "TSOH", "PTSA·H₂O", "HCl",'PTSA','Benzoic acid','[BSMIm]HSO4','TFSA','Benzoic anhydride'
         ,'Sc(OTf)3','Yb(OTf)3','sodium oleyl sulfate','Formaldehyde']
basic = ["KOH", "Et3N", "EtONa", "DBU", "Pyridine", "Piperidine", "NaOH",'DBU','MeOLi','MeONa','BuOK','Cs2CO3','K2CO3','Isoquinoline','Pyrrolidine']

# 文件路径
file_path = '5-20250414-sum_data.xlsx'

# 加载数据

references = pd.read_excel('data.xlsx')
solvent_props = pd.read_excel(file_path, sheet_name='solvent properties').set_index('Solvent')
col=solvent_props.columns
index=solvent_props.index
scaler1 = StandardScaler()
y1= scaler1.fit_transform(solvent_props)
solvent_props=pd.DataFrame(y1,columns=col)

# 去除溶剂名称中的空格
solvent_props.index = index.str.strip()
# ----------------------------
# 特征工程
# ----------------------------
# 创建 Morgan 指纹生成器
n_bits = 512
morgan_generator = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=n_bits)
df=references.copy()
name1='807-19-2'
name2='108-78-1'
r=main_fun(name1,name2,model,model1,model2,model3,model4,model5,df,info_dict)
r.to_excel('r.xlsx',index=False)
r

807-19-2
https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=json&query={%22select%22:%22*%22,%22collection%22:%22compound%22,%22order%22:[%22relevancescore,desc%22],%22start%22:1,%22limit%22:10,%22where%22:{%22ands%22:[{%22*%22:%22807-19-2%22}]},%22width%22:1000000,%22listids%22:0}
108-78-1
https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=json&query={%22select%22:%22*%22,%22collection%22:%22compound%22,%22order%22:[%22relevancescore,desc%22],%22start%22:1,%22limit%22:10,%22where%22:{%22ands%22:[{%22*%22:%22108-78-1%22}]},%22width%22:1000000,%22listids%22:0}
807-19-2
https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=json&query={%22select%22:%22*%22,%22collection%22:%22compound%22,%22order%22:[%22relevancescore,desc%22],%22start%22:1,%22limit%22:10,%22where%22:{%22ands%22:[{%22*%22:%22807-19-2%22}]},%22width%22:1000000,%22listids%22:0}
108-78-1
https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=json&query={%22select%2

Solvent1  Solvent1_amount Solvent2 Solvent2_amount Solvent3  \
0        DMAc             3.25                                     
1     Dioxane             3.38                                     
2      n-BuOH             2.54                                     
3        MeOH             2.87     DMAc             2.7            
4      n-BuOH             3.36      THF            3.54            
5        DMAc             3.35      DMF            4.42            
6        EtOH             3.89                                     
7       O-DCB             3.36                                     
8        DMAc             3.53                                     
9  Mesitylene             2.80                                     

  Solvent3_amount Solvent_ratio  Ligand1_amount  Ligand2_amount  Ligand_ratio  \
0                                          1.22            1.39          0.88   
1                                          0.65            2.08          0.31   
2                                          1.47           15.84          0.09   
3                          1.06            1.09            5.71          0.19   
4                          0.95            5.93           18.48          0.32   
5                          0.76            1.22            2.35          0.52   
6                                          0.43            2.41          0.18   
7                                          4.80            4.77          1.01   
8                                          6.03           17.96          0.34   
9                                          0.77            1.81          0.43   

  Additive_name    Temp   Time  
0          none  398.01  91.62  
1          none  398.13  91.55  
2          none  398.02  91.75  
3          none  398.23  91.39  
4          none  398.83  92.15  
5          none  411.67  91.73  
6          none  431.43  91.41  
7          none  422.27  91.42  
8          none  398.32  88.51  
9          none  398.83  88.91

In [ ]:
https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=json&query={%22select%22:%22*%22,%22collection%22:%22compound%22,%22order%22:[%22relevancescore,desc%22],%22start%22:1,%22limit%22:10,%22where%22:{%22ands%22:[{%22*%22:%22807-19-2%22}]},%22width%22:1000000,%22listids%22:0}

In [22]:
url = (
    f'https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=json'
    f'&query={{%22select%22:%22*%22,%22collection%22:%22compound%22,'
    f'%22order%22:[%22relevancescore,desc%22],%22start%22:1,%22limit%22:10,'
    f'%22where%22:{{%22ands%22:[{{%22*%22:%22{name1}%22}}]}},'
    f'%22width%22:1000000,%22listids%22:0}}'
)
url

'https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=json&query={%22select%22:%22*%22,%22collection%22:%22compound%22,%22order%22:[%22relevancescore,desc%22],%22start%22:1,%22limit%22:10,%22where%22:{%22ands%22:[{%22*%22:%22807-19-2%22}]},%22width%22:1000000,%22listids%22:0}'

In [18]:
url

'https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=json&query={"select":"*","collection":"compound","order":["relevancescore,desc"],"start":1,"limit":10,"where":{"ands":[{"*":"807-19-2"}]},"width":1000000,"listids":0}'